In [1]:
# Import Dependencies
import psycopg2
import pandas as pd

# Import postgres server info key and BEA API Key
from postgres_info import user, password

In [ ]:
# Import data into pandas dataframe
global_warming_df = pd.read_csv('Resources/global_warming_dataset.csv')
global_warming_df.head()

,Country,Year,Temperature_Anomaly,CO2_Emissions,Population,Forest_Area,GDP,Renewable_Energy_Usage,Methane_Emissions,Sea_Level_Rise,...,Waste_Management,Per_Capita_Emissions,Industrial_Activity,Air_Pollution_Index,Biodiversity_Index,Ocean_Acidification,Fossil_Fuel_Usage,Energy_Consumption_Per_Capita,Policy_Score,Average_Temperature
0,Country_103,1913,-1.163537,8.876061e+08,1.627978e+08,54.872178,6.139887e+12,76.710013,8.317626e+06,8.111839,...,82.691409,2.285351,4.060975,150.285539,90.073356,8.025470,39.163860,1480.164332,78.870012,20.825292
1,Country_180,1950,-0.432122,4.497517e+08,4.281359e+08,84.051006,2.601447e+12,68.450021,6.206540e+06,42.025915,...,59.322883,17.411668,85.300604,27.305922,88.289837,8.021719,28.252554,1482.730048,32.600905,28.720587
2,Country_93,2014,0.444954,4.579080e+08,4.926732e+08,72.295357,5.192677e+12,36.725699,1.056885e+06,20.953840,...,94.982931,12.039703,83.804880,216.911429,86.936256,7.647408,61.548382,706.918809,37.671300,15.014084
3,Country_15,2020,-1.171616,5.049503e+08,1.252169e+09,17.259684,8.252128e+12,77.547901,1.986813e+06,45.599595,...,62.064250,2.853957,47.014265,35.869182,44.904331,7.569353,82.423750,2616.238324,86.581725,-1.277086
4,Country_107,1964,-0.564038,6.898891e+08,2.932960e+08,44.438605,8.560746e+12,10.019576,3.313252e+06,7.652150,...,84.431279,19.801173,89.379613,284.263093,8.102916,8.015415,29.964450,4975.683780,20.618406,2.861989


In [ ]:
# Convert dataframe values to a list
global_warming_list = global_warming_df.values.tolist()

In [4]:
# Use a 'try' statement to handle errors connecting to the database
try:
    
    # Connect to the database
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=user,
        password=password,
        database="postgres"
    )
    
    # Make sure autocommit is 'True' to commit new database successfully
    conn.autocommit = True

# Use 'except' statement to handle and display an error connect to PostgreSQL
except psycopg2.Error as e:
    print('-'*60)
    print("Error connecting to the database:")
    print(e)

# Use 'else' statemnt is connection if successful
else:
    print('-'*60)
    print("Connection established successfully")

# Create cursor
cur = conn.cursor()

# Query to create 'ev_db' database
sql = '''CREATE DATABASE global_warming_db;'''

# Use 'try' statement to execute above SQL statement
print('-'*60)
try:
    cur.execute(sql)
    print("Database successfully created!")

# Use 'except' statement to handle if database already exists
except:
    print("Database already exists! Execute command aborted...")

# Close cursor
cur.close()

# Close connection
conn.close()

------------------------------------------------------------
Connection established successfully
------------------------------------------------------------
Database already exists! Execute command aborted...


In [5]:
# Use a 'try' statement to handle errors connecting to the database
try:

    # Connect to the database
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=user,
        password=password,
        database="global_warming_db"
    )

# Use 'except' statement to handle and display an error connect to PostgreSQL
except psycopg2.Error as e:
    print('-'*60)
    print("Error connecting to the database:")
    print(e)

# Use 'else' statemnt if connection is successful
else:
    print('-'*60)
    print("Connection re-established successfully")

# Create cursor
cur = conn.cursor() 

# SQL queries to drop tables if already exists. Use 'CASCADE' to ensure all dependent tables are dropped
drop_table_query = '''DROP TABLE IF EXISTS cafv_eligibility CASCADE;'''

# SQL queries to create tables for our data
create_table_query = '''
    CREATE TABLE global_warming_data (
        record_id SERIAL PRIMARY KEY,
        country VARCHAR(12),
        year INT,
        temperature_anomaly FLOAT,
        co2_emissions FLOAT,
        population FLOAT,
        forest_area FLOAT,
        gdp FLOAT,
        renewable_energy_usage FLOAT,
        methane_emissions FLOAT,
        sea_level_rise FLOAT,
        artic_ice_extent FLOAT,
        urbanization FLOAT,
        deforestation_rate FLOAT,
        extreme_weather_events INT,
        average_rainfall FLOAT,
        solar_energy_potential FLOAT,
        waste_management FLOAT,
        per_capita_emissions FLOAT,
        industrial_activity FLOAT,
        air_pollution_index FLOAT,
        biodiversity_index FLOAT,
        ocean_acidification FLOAT,
        fossil_fuel_usage FLOAT,
        energy_consumption_per_capita FLOAT,
        policy_score FLOAT,
        average_temperature FLOAT
    );
    '''

# Execute Drop table queries
cur.execute(drop_table_query)

# Execute the CREATE TABLE queries
cur.execute(create_table_query)
print('-'*60)
print("Table created or recreated successfully!")

------------------------------------------------------------
Connection re-established successfully
------------------------------------------------------------
Table created or recreated successfully!


In [ ]:
# Print statement showing data is loading
print('-'*60)
print('Loading global warming data into table...')

# Use a for loop to insert each row of data into main vehicle table
for row in global_warming_list:
    sql = '''INSERT INTO global_warming_data 
    (country, year, temperature_anomaly, co2_emissions, population, forest_area, gdp, renewable_energy_usage, methane_emissions, 
    sea_level_rise, artic_ice_extent, urbanization, deforestation_rate, extreme_weather_events, average_rainfall, solar_energy_potential, 
    waste_management, per_capita_emissions, industrial_activity, air_pollution_index, biodiversity_index, ocean_acidification, fossil_fuel_usage, 
    energy_consumption_per_capita, policy_score, average_temperature) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s);'''
    cur.execute(sql, row)

# Print statment showing data loaded successfully
print('Gloabal Warming data added successfully!')

# Commit all the changes to the database
conn.commit()

# Close the cursor
cur.close()

# Close the connection
conn.close()

------------------------------------------------------------
Loading global warming data into table...
Gloabal Warming data added successfully!
